In [ ]:
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken
!pip install PyPDF2
!pip install pypdf
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
import chromadb
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
import os
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationalRetrievalChain
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
import pickle
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
pdf_docs = [r'/content/drive/MyDrive/Langchain/The-Theory-of-Everything-pdf-free-download.pdf',
           r'/content/drive/MyDrive/Langchain/Stephen Hawking -A Brief History Of Time.pdf']

**Without taking source into consideration**

In [ ]:
def get_pdf(pdf_docs):
  text = ''
  for pdf in pdf_docs:
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
      text += page.extract_text()
  return text


In [ ]:
def get_text_chunks(text):
  text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
  chunks = text_splitter.split_text(text)
  return chunks

In [ ]:
def get_vectordb(chunks):
  embedding_function = OpenAIEmbeddings()
  db = FAISS.from_texts(chunks, embedding_function)
  return db

In [ ]:
text = get_pdf(pdf_docs)
chunks = get_text_chunks(text)
vector_db = get_vectordb(chunks)

In [ ]:
len(chunks)

804

In [ ]:
chunks[5]

'IDEAS ABOUT THE UNIVERSE       5F9A4512-DBD2-4640-B53D-F3CF22FFE0A2As long ago as 340 B.C. Aristotle, in his book On the Heavens , was able to\nput forward two good arguments for believing that the Earth was a round\nball rather than a flat plate. First, he realized that eclipses of the moon werecaused by the Earth coming between the sun and the moon. The Earth’s shad-ow on the moon was always round, which would be true only if the Earth wasspherical. If the Earth had been a flat disk, the shadow would have been elon-gated and elliptical, unless the eclipse always occurred at a time when the sunwas directly above the center of the disk.\nSecond, the Greeks knew from their travels that the Pole Star appeared lower'

In [ ]:
with open('/content/drive/MyDrive/Langchain/stephawk_db.pkl', 'wb') as f:
  pickle.dump(vector_db, f)

In [ ]:
with open(r'/content/drive/MyDrive/Langchain/stephawk_db.pkl', 'rb') as f:
  toe_db1 = pickle.load(f)

In [ ]:
llm = ChatOpenAI()
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
def Answer(query):
  conversation_chain = ConversationalRetrievalChain.from_llm(llm = llm, memory = memory, retriever = toe_db1.as_retriever())
  ans = conversation_chain({'question': query}, return_only_outputs = True)
  return ans['answer']

In [ ]:
query = 'Tell me about anthropic principle'
Answer(query)

'There are several objections and controversies surrounding the strong anthropic principle. One objection is that it is unclear in what sense all these different universes can be said to exist. If they are truly separate from each other, then what happens in another universe would have no observable consequences in our own universe. Another objection is that if these universes are just different regions of a single universe, then the laws of science would have to be the same in each region, which would reduce the strong anthropic principle to the weak one.\n\nAnother objection to the strong anthropic principle is that it goes against the history of science, which has moved away from the idea that the universe exists solely for our sake. Our understanding of the universe has evolved from geocentric cosmologies to the modern picture of a vast and diverse universe with countless galaxies. The idea that this entire construction exists only for our benefit is difficult to believe.\n\nThese 

**Taking source into consideration**

In [ ]:
def get_pdf2(pdf_docs):
  pages = []
  for pdf in pdf_docs:
    pdf_reader = PyPDFLoader(pdf)
    for page in pdf_reader.load_and_split():
      pages.append(page)
  return pages

In [ ]:
def get_text_chunks2(text):
  text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
  chunks = text_splitter.split_documents(text)
  return chunks

In [ ]:
def get_vectordb2(chunks):
  embedding_function = OpenAIEmbeddings()
  db = FAISS.from_documents(chunks, embedding_function)
  return db

In [ ]:
text = get_pdf2(pdf_docs)
chunks = get_text_chunks2(text)
vector_db = get_vectordb2(chunks)

In [ ]:
len(chunks)

943

In [ ]:
chunks[5]

Document(page_content="In the third lecture I shall talk about black holes. These are formed when a\nmassive star or an even larger body collapses in on itself under its own gravitational pull. According to Einstein’s general theory of relativity, anyone foolish enough to fall into a black hole will be lost forever. They will not beable to come out of the black hole again. Instead, history, as far as they are concerned, will come to a sticky end at a singularity. However, general relativity is a classical theory—that is, it doesn't take into account the uncertainty principle of quantum mechanics.INTRODUCTION\nIV       5F9A4512-DBD2-4640-B53D-F3CF22FFE0A2", metadata={'source': '/content/drive/MyDrive/Langchain/The-Theory-of-Everything-pdf-free-download.pdf', 'page': 7})

In [ ]:
with open('/content/drive/MyDrive/Langchain/stephawk_db2.pkl', 'wb') as f:
  pickle.dump(vector_db, f)

In [ ]:
with open(r'/content/drive/MyDrive/Langchain/stephawk_db2.pkl', 'rb') as f:
  toe_db2 = pickle.load(f)

In [ ]:
llm = ChatOpenAI()
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
def Answer2(query):
  conversation_chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever = toe_db2.as_retriever())
  ans = conversation_chain({'question': query}, return_only_outputs = True)
  return ans

In [ ]:
query = 'Tell me about anthropic principle'
Answer2(query)

{'answer': 'The anthropic principle is a concept that suggests that the conditions necessary for the development of intelligent life will only be met in certain regions of a universe that is large or infinite in space and/or time. This principle can be divided into the weak anthropic principle and the strong anthropic principle. The weak anthropic principle states that intelligent beings in these regions should not be surprised if they observe that their locality in the universe satisfies the conditions necessary for their existence. The strong anthropic principle, on the other hand, claims that the entire vast construction of the universe exists for the sake of intelligent life. However, there are objections to the strong anthropic principle, including the question of the existence of multiple universes and the consistency of the laws of science across different regions. The strong anthropic principle would also run against the history of scientific discoveries that have shown the vas

In [ ]:
query = 'Tell me about the thermodymanic arrow of time'
Answer2(query)

{'answer': 'The thermodynamic arrow of time is the concept that the entropy, or disorder, of a closed system tends to increase over time. This asymmetry in the behavior of physical systems gives rise to the arrow of time, where certain processes are irreversible and occur in a particular direction. It is closely linked to the concept of causality, as events in the past have causal effects on events in the future but not vice versa. The thermodynamic arrow of time is a macroscopic phenomenon and emerges due to the statistical nature of thermodynamics in large-scale systems. It is important to note that the thermodynamic arrow of time is essentially the same as the psychological arrow of time, and they both point in the same direction. The existence of a well-defined thermodynamic arrow of time is predicted by the no boundary proposal for the universe. At the microscopic level, the behavior of particles is typically time-reversible, but on a larger scale, the arrow of time is observed du

In [ ]:
query = 'What did Aristotle say about the universe?'
Answer2(query)

{'answer': 'Aristotle believed that all matter in the universe was made up of four basic elements - earth, air, fire, and water. He also believed that the Earth was stationary and that the sun, moon, planets, and stars moved in circular orbits around the Earth because he believed that the Earth was the center of the universe and that circular motion was the most perfect. Additionally, Aristotle and other Greek philosophers did not believe in the idea of a creation and believed that the human race and the world around it had existed forever.\n',
 'sources': '/content/drive/MyDrive/Langchain/Stephen Hawking -A Brief History Of Time.pdf, /content/drive/MyDrive/Langchain/The-Theory-of-Everything-pdf-free-download.pdf'}

In [ ]:
query = 'What is the theory of relativity?'
Answer2(query)

{'answer': 'The theory of relativity states that the laws of science should be the same for all freely moving observers, regardless of their speed. It also suggests that there is no unique absolute time, but rather each individual has their own personal measure of time based on their location and motion. Additionally, the theory of relativity changes our understanding of space and time, revealing that they are not completely separate but rather combined to form a concept known as space-time. In the theory of general relativity, the strength of gravity and the value of the cosmological constant are the only adjustable quantities. \n',
 'sources': '/content/drive/MyDrive/Langchain/Stephen Hawking -A Brief History Of Time.pdf, /content/drive/MyDrive/Langchain/The-Theory-of-Everything-pdf-free-download.pdf'}

In [ ]:
query = 'What is the difference of ideolody between Newton and Ariatotle and Galileo?'
Answer(query)

"The main difference in ideology between Newton, Aristotle, and Galileo is their views on motion and the natural state of objects. \n\nAristotle believed in a preferred state of rest, where any body would naturally be at rest unless it was driven by a force or impulse. He also believed that the Earth was at rest. \n\nGalileo challenged Aristotle's beliefs and conducted experiments to show that objects of different weights fall at the same rate, contradicting Aristotle's belief that heavier objects fall faster. Galileo also supported the Copernican theory, which proposed that the Earth and planets orbit the sun, challenging the geocentric model supported by Aristotle.\n\nNewton's ideology built upon the work of Galileo and Copernicus. He developed the laws of motion and the law of universal gravitation. Newton's laws of motion stated that objects in motion will remain in motion unless acted upon by an external force, and that the rate of change of momentum of an object is directly propo

In [ ]:
memory.buffer

[HumanMessage(content='Tell me about anthropic principle'),
 AIMessage(content="The anthropic principle is a concept in cosmology and philosophy that suggests that the observed state of the universe is influenced by the existence of intelligent beings like ourselves. There are two versions of the anthropic principle: the weak and the strong.\n\nThe weak anthropic principle states that intelligent life can only exist in certain regions of the universe that satisfy the necessary conditions for its development. This principle suggests that we should not be surprised to find ourselves in a locality that supports our existence, just as a wealthy person living in a wealthy neighborhood wouldn't be surprised by the absence of poverty. One example of the weak anthropic principle being used is to explain why the Big Bang occurred around ten billion years ago, as it takes that long for intelligent beings to evolve.\n\nThe strong anthropic principle takes the idea further and suggests that the un